# Bulge data exploration

In Part A, we showed how to ingest raw DECam images and to process them. A processed data set is available at `/project/stack-club/course_data/DECAM_BULGE`. In this notebook we will use the Butler to explore the processed data. 



In [ ]:
# Make plots available to the notebook
%matplotlib inline

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import lsst.daf.persistence as dafPersist
import lsst.afw.table as afwTable
import lsst.afw.display as afwDisplay
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
from lsst.meas.algorithms import SourceDetectionTask, SourceDetectionConfig
from lsst.meas.deblender               import SourceDeblendTask, SourceDeblendConfig
from lsst.meas.base                    import SingleFrameMeasurementTask, SingleFrameMeasurementConfig
from lsst.meas.base import NoiseReplacer, NoiseReplacerConfig
import matplotlib
import matplotlib.animation as animation
from matplotlib.colors import LogNorm
from IPython.display import IFrame, display, Markdown, HTML
import lsst.meas.extensions.photometryKron
import lsst.geom as lsstGeom

afwDisplay.setDefaultBackend('matplotlib') 

matplotlib.rcParams["figure.figsize"] = (8, 8)
matplotlib.rcParams["font.size"] = 12
matplotlib.rcParams["figure.dpi"] = 600


In [ ]:
# Repo directory
REPO_DIR = '/project/stack-club/course_data/DECAM_BULGE/' 


## After processing the images we can start with the butler and displaying images:

In [ ]:
butler = dafPersist.Butler(RERUN_DIR)

In [ ]:
rowData = []
calexp_list = []
exparray_list = []
visit_data_list = []
metadata = butler.queryMetadata('calexp',['visit','ccd','filter'])
for dataset in metadata:
    dataId = {'visit': int(dataset[0]), 'ccd': int(dataset[1]), 'filter':dataset[2]}
    if butler.datasetExists('src', dataId=dataId):
        srcCatalog = butler.get('src', dataId=dataId).asAstropy().to_pandas() # get the source catalog
        raMax = 0#srcCatalog['coord_ra'].max()
        raMin = 0#srcCatalog['coord_ra'].min()
        decMax = 0#srcCatalog['coord_dec'].max()
        decMin = 0#srcCatalog['coord_dec'].min()
        raCenter = 0#0.5*(raMax + raMin)
        decCenter = 0#0.5*(decMax + decMin)
        rowData.append([int(dataset[0]), int(dataset[1]), dataset[2], raCenter, decCenter, 
                  raMin, raMax, decMin, decMax])
        
        if int(dataset[1]) == 1:
            calexp = butler.get('calexp', visit=int(dataset[0]), ccd=1)
            calexp_list.append( calexp )
            exparray_list.append( calexp.maskedImage.image.getArray() )
            exp_visit_info = calexp.getInfo().getVisitInfo()
            visit_date = exp_visit_info.getDate()
            visit_data_list.append( [int(dataset[0]), visit_date.toPython()] )
            
        
df_valid_visists = pd.DataFrame(rowData, columns=['visit', 'ccd', 'DECAM_filter', 'ra_center', 'dec_center', 
                                        'ra_min', 'ra_max', 'dec_min', 'dec_max'])

df_visit_data = pd.DataFrame(visit_data_list, columns=['visit', 'date'])

In [ ]:
df_valid_visists.head()

In [ ]:
df_visit_data

In [ ]:
parquet_save_path = '/project/stack-club/course_data/DECAM_BULGE'
df_valid_visists.to_parquet( os.path.join(parquet_save_path,'df_valid_visits.parquet.gzip'), compression='gzip')


In [ ]:
df_valid_visists.visit.unique()

In [ ]:
CCD = 1

for ii,calexp in enumerate(calexp_list[:10]):

    plt.figure(ii)
    display = afwDisplay.Display(frame=ii, backend='matplotlib')
    display.scale("linear", "zscale")
    display.mtv(calexp[500:1500,2500:3000])


TODO:

- Check astrometry
- Make light curves
- create co-add image
- run ap-pipe

